In [55]:
import pandas as pd
import numpy as np
import tensorly as ts
import urllib3
import json

In [56]:
# Dataframe configs
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [57]:
# Setting SonarCloud.io project key
#chave = 'fga-eps-mds_2020.1-VC_Gestor-BackEnd:packages&branch=master'

frontend_key = 'fga-eps-mds_2020.1-VC_Gestor-FrontEnd'
backend_key = 'fga-eps-mds_2020.1-VC_Gestor-BackEnd'

In [58]:
# Defining the raw data extract from sonar cloud
metrics_list = ['files',
               'functions',
               'complexity',
               'ncloc',
               'comment_lines',
               'comment_lines_density',
               'duplicated_lines_density',
               'test_success_density',
               'coverage',
               'bugs',
               'reliability_rating',
               'vulnerabilities',
               'security_rating',
               'open_issues',
               'sqale_rating']

In [59]:
print(len(metrics_list))

15


In [60]:
# Assembling the metric list string and API request URLs

def generate_metric_string(metric_list):
    metric_str = ''

    for metric in metric_list:
        metric_str += metric + ','

    return metric_str

In [61]:
metric_str = generate_metric_string(metrics_list)

"\nstring_metricKeys = ''\nfor metric in metrics_list:\n    string_metricKeys += metric + ','\n"

In [62]:
# Defining URLs

frontend_url = f"https://sonarcloud.io/api/measures/component_tree?component={frontend_key}&metricKeys={metric_str}&ps=500"
backend_url = f"https://sonarcloud.io/api/measures/component_tree?component={backend_key}&metricKeys={metric_str}&ps=500"

In [63]:
# Starting connection
http = urllib3.PoolManager()

In [64]:
# Sonar Cloud API Request
frontend_request = http.request('GET', frontend_url)
print(frontend_request.status)

backend_request = http.request('GET', backend_url)
print(backend_request.status)

200
200


In [65]:
# Decoding json files extracted from sonar
frontend_json = json.loads(frontend_request.data.decode('utf-8'))
backend_json = json.loads(backend_request.data.decode('utf-8'))

In [66]:
'''
# Saving json files locally
file_name = 'fga-eps-mds_2020.1-VC_Gestor-BackEnd-R2.json'

with open(file_name, 'w') as outfile:
    json.dump(json_repository, outfile)
'''

def metric_per_file(json):
    file_json = []

    for component in json['components']:
        if component['qualifier'] == 'FIL':
            file_json.append(component)

    return file_json

In [67]:
frontend_file_json = metric_per_file(frontend_json)
backend_file_json = metric_per_file(backend_json)

In [68]:
def generate_file_dataframe(metric_list, json, language_extension):
    df_columns = metric_list
    df = pd.DataFrame(columns = df_columns)

    for file in json:
        try:
            if file['language'] == language_extension:
                for measure in file['measures']:
                    df.at[file['path'], measure['metric']] = measure['value']
        except:
            pass

    df.reset_index(inplace = True)
    df.drop(['files'], axis=1)
    df = df.rename({'index': 'path'}, axis=1)

    return df

In [69]:
frontend_files_data = generate_file_dataframe(metrics_list, frontend_file_json, language_extension = 'js')
backend_files_data = generate_file_dataframe(metrics_list, backend_file_json, language_extension = 'js')

In [70]:
# FRONTEND METRICS DATA PER FILE

frontend_files_data

,path,files,functions,complexity,ncloc,comment_lines,comment_lines_density,duplicated_lines_density,test_success_density,coverage,bugs,reliability_rating,vulnerabilities,security_rating,open_issues,sqale_rating
0,src/components/404/404.js,1,1,1,14,0,0.0,0.0,100.0,0.0,0,1.0,0,1.0,2,1.0
1,src/components/pages/Anuncio.js,1,5,7,120,13,9.8,10.9,100.0,0.0,0,1.0,0,1.0,13,1.0
2,src/services/apiBeneficio.js,1,0,0,5,0,0.0,0.0,100.0,0.0,0,1.0,0,1.0,0,1.0
3,src/services/apiNoticias.js,1,0,0,5,0,0.0,0.0,100.0,100.0,0,1.0,0,1.0,0,1.0
4,src/services/apiPostagem.js,1,0,0,5,0,0.0,0.0,100.0,100.0,0,1.0,0,1.0,0,1.0
5,src/services/apiUser.js,1,0,0,5,0,0.0,0.0,100.0,0.0,0,1.0,0,1.0,0,1.0
6,src/components/app/App.test.js,1,1,1,8,0,0.0,0.0,100.0,0.0,0,1.0,0,1.0,0,1.0
7,src/components/pages/BeneficiosCriar.js,1,1,1,20,0,0.0,0.0,100.0,0.0,0,1.0,0,1.0,2,1.0
8,src/components/pages/BeneficiosEditar.js,1,1,1,20,0,0.0,0.0,100.0,0.0,0,1.0,0,1.0,0,1.0
9,src/components/pages/BeneficiosGerenciar.js,1,8,10,53,3,5.4,0.0,100.0,0.0,0,1.0,0,1.0,8,1.0


In [71]:
# BACKEND METRICS DATA PER FILE

backend_files_data


,path,files,functions,complexity,ncloc,comment_lines,comment_lines_density,duplicated_lines_density,test_success_density,coverage,bugs,reliability_rating,vulnerabilities,security_rating,open_issues,sqale_rating
0,packages/user/src/controller/AuthenticateUserS...,1,1,1,11,1,8.3,0.0,100.0,0.0,0,1.0,0,1.0,0,1.0
1,packages/user/src/controller/AuthResolve.js,1,1,3,22,1,4.3,0.0,100.0,100.0,0,1.0,0,1.0,0,1.0
2,packages/benefits/src/models/benefit.js,1,0,0,30,0,0.0,0.0,100.0,100.0,0,1.0,0,1.0,0,1.0
3,packages/benefits/src/controller/BenefitContro...,1,5,5,44,18,29.0,0.0,100.0,28.0,0,1.0,0,1.0,0,1.0
4,packages/benefits/src/test/BenefitsController....,1,12,12,251,0,0.0,0.0,100.0,100.0,0,1.0,0,1.0,0,1.0
5,packages/user/src/controller/ChangePasswordRes...,1,1,5,29,0,0.0,0.0,100.0,100.0,0,1.0,0,1.0,0,1.0
6,packages/user/src/controller/CheckCodeResolve.js,1,1,4,23,0,0.0,0.0,100.0,100.0,0,1.0,0,1.0,0,1.0
7,packages/news/src/test/ControllerTesting.spec.js,1,13,13,316,0,0.0,0.0,100.0,100.0,0,1.0,0,1.0,0,1.0
8,packages/user/src/test/ControllerTesting.spec.js,1,24,24,575,0,0.0,33.4,100.0,100.0,0,1.0,0,1.0,0,1.0
9,packages/benefits/src/controller/CreateBenefit...,1,1,4,16,0,0.0,0.0,100.0,100.0,0,1.0,0,1.0,0,1.0


In [72]:
#printing CC per file descriptive statistics
backend_files_data['complexity'].astype(int).describe()

count    31.000000
mean      4.129032
std       4.856212
min       0.000000
25%       1.500000
50%       3.000000
75%       5.000000
max      24.000000
Name: complexity, dtype: float64

In [73]:
backend_files_data['complexity/functions']=backend_files_data['complexity'].astype(float)/backend_files_data['functions'].astype(float)

#printing the average CC per function descriptive statistics
backend_files_data['complexity/functions'].describe()

count    26.000000
mean      2.384615
std       1.940658
min       1.000000
25%       1.000000
50%       2.000000
75%       3.000000
max       9.000000
Name: complexity/functions, dtype: float64

In [74]:
# Defining metrics for measure em1, where em1 = density of non-complex file
m0 = np.median(backend_files_data['complexity/functions'])
m1 = backend_files_data['complexity'].astype(float)
m2 = backend_files_data['functions'].astype(float)
m3 = len(backend_files_data)

#  intervals for em1 interpretation
x = np.array([0, m0])
y = np.array([1, 0])

# Defining intervals for em1 interpretation
IF1 = np.interp(list(((m1/m2) < m0)[(m2 > 0)]),x, y)

em1 = sum(IF1)/m3

em1

0.8387096774193549

In [75]:
# Defining metrics for measure em2, where em2 = density of commented files
m4 = backend_files_data['comment_lines_density'].astype(float)

#  intervals for em1 interpretation
x = np.array([0.1, 0.3])
y = np.array([1, 0])

#(m4 >= 0.1 and m4 <= 0.3)

# Defining intervals for em2 interpretation
em2i = []
for if2i in m4:
    if 10 <= if2i <= 30:
        em2i.append(np.interp(if2i/100,x, y))
    else:
        em2i.append(0)

em2 = np.sum(em2i)/m3

em2

0.027258064516129034

In [76]:
# Defining metrics for measure ma3,where ma3 = absence of duplication on file (density)
m5 = backend_files_data['duplicated_lines_density'].astype(float)

#  intervals for em1 interpretation
x = np.array([0.1, 0.2])
y = np.array([1, 0])

# Intervals for ma3 interpretation
em3i = []
for if3i in m5:
    if 10 <= if3i <= 20:
        em3i.append(np.interp(if3i/100,x, y))
    else:
        em3i.append(0)

em3 = np.sum(em3i)/m3

em3

0.0

In [77]:
# Defining  Modifiability quality sub-characteristic tensor.
# wem1 = 33%
# wem2 = 33%
# wem3 = 33%

wem1=wem2=wem3 = 0.3374

SC_Modifiability_EM_Weights = np.array([wem1, wem2, wem3]).reshape(1,3,1)
SC_Modifiability_EM = np.array([em1, em2, em3]).reshape(1,3,1)


T_SC_Modifiability = ts.tensor(SC_Modifiability_EM)


print("Modifiability Tensor" + '\n' + str(T_SC_Modifiability))
print(T_SC_Modifiability.ndim)

Modifiability Tensor
[[[0.83870968]
  [0.02725806]
  [0.        ]]]
3


In [78]:
# Weighting Modifiability tensor at measure level.

for i in range(T_SC_Modifiability.ndim):
     T_SC_Modifiability[::T_SC_Modifiability.ndim][0][i-1] = \
         ts.kron(T_SC_Modifiability[::T_SC_Modifiability.ndim][0][i-1] ,
                 SC_Modifiability_EM_Weights[::T_SC_Modifiability.ndim][0][i-1])

print("Weighted SC_Modifiability Tensor measures" + '\n' + str(T_SC_Modifiability))

Weighted SC_Modifiability Tensor measures
[[[0.28298065]
  [0.00919687]
  [0.        ]]]


In [79]:
# Weighting Modifiability tensor at sub-characteristic level.
# wsc1 = 100%

wsc1 = 1

T_SC_Modifiability = ts.kron(T_SC_Modifiability,wsc1)

print("Weighted SC_Modifiability Tensor" + '\n' + str(T_SC_Modifiability))

Weighted SC_Modifiability Tensor
[[[0.28298065]
  [0.00919687]
  [0.        ]]]


In [80]:
# Defining the maintainability quality characteristic tensor.

T_C_Maintainability = np.array([])
T_C_Maintainability = np.append(T_C_Maintainability,T_SC_Modifiability)

# Weighting Maintainability tensor at characteristic level.
# wc1 = 100%

wc1 = 1

T_C_Maintainability = ts.kron(T_C_Maintainability,wc1)

# Weighted C_Maintainability Tensor
print(str(T_C_Maintainability.reshape(1,3,1)))

[[[0.28298065]
  [0.00919687]
  [0.        ]]]


In [81]:
# Aggregating C_Maintainability Tensor. The aggregation view is obtained from Frobenius norm of tensor.
print(str(ts.norm(T_C_Maintainability)))

0.28313005487142723


In [82]:
SQC_1 = np.array([])
SQC_1 = np.append(SQC_1, T_C_Maintainability)

# The overall quality at release 1 in VC_Gestor product.
# Github Repository: https://github.com/fga-eps-mds/2020.1-VC_Gestor-BackEnd
print(str(ts.norm(SQC_1)))

0.28313005487142723
